In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import word_tokenize
from tqdm import tqdm

In [0]:
base = 'drive/My Drive/Kaggle/Quora Insincere Questions Classification/'

In [0]:
df = pd.read_csv(base + 'data.csv')
df.head(3)

qid  ... target
0  00002165364db923c7e6  ...      0
1  000032939017120e6e44  ...      0
2  0000412ca6e4628ce2cf  ...      0

[3 rows x 3 columns]

In [0]:
df_sort = df.sort_values(['target', 'qid'])
df_sort.head(3)

qid  ... target
0  00002165364db923c7e6  ...      0
1  000032939017120e6e44  ...      0
2  0000412ca6e4628ce2cf  ...      0

[3 rows x 3 columns]

In [0]:
df_clean = pd.concat([df_sort[:1000], df_sort[-1000:]])


In [0]:
corpus = df_clean['question_text'].to_numpy()

# vectorizer = CountVectorizer()
# X = vectorizer.fit_transform(corpus)
# X

In [0]:
import io

def load_vectors(fname):
    fin = io.open(fname, 'r', encoding='utf-8', newline='\n', errors='ignore')
    n, d = map(int, fin.readline().split())
    data = {}
    for line in tqdm(fin):
        tokens = line.rstrip().split(' ')
        data[tokens[0]] = list(map(float, tokens[1:]))
    return data
word_vectors = load_vectors(base + 'wiki-news-300d-1M/wiki-news-300d-1M.vec') 

999994it [01:43, 9706.07it/s] 


In [0]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [0]:
max_len = 0
curr_id = 0
id_list = []
feature_list = []

for sentence in tqdm(corpus):
    sentence_vector = []
    tokens = word_tokenize(sentence)
    for word in tokens:
        try:
            vec = word_vectors[word]
        except KeyError:
            continue
        sentence_vector.append(vec)
        
    feature_list.append(sentence_vector)
    curr_len = len(sentence_vector)
    curr_id += curr_len
    id_list.append(curr_id)
    
    max_len = max(max_len, curr_len)
    

100%|██████████| 2000/2000 [00:00<00:00, 4807.61it/s]


In [0]:
target = df_clean['target'].to_numpy()

In [0]:
from google.colab import files

# f = open(base + "wongnai_word2vec_dataset.csv", "a")

with open(base + "quora_word2vec_dataset.csv", 'w') as f:
    # header
    for i in range(300):
        f.write('{},'.format(i))
    f.write('target\n')

    # a sentence vector
    target_id = 0
    for sentence in tqdm(feature_list):
        length = len(sentence)
        
        # padding
        for i in range(max_len - length):
            for j in range(300):
                f.write('{},'.format(0))
            f.write('{}\n'.format(target[target_id]))

        for word in sentence:
            for val in word:
                f.write('{},'.format(val))
            f.write('{}\n'.format(target[target_id]))

        # increase index of target
        target_id += 1

In [0]:
files.download(base + "quora_word2vec_dataset.csv")